Example from here: https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/deployment/deploy-to-cloud/model-register-and-deploy.ipynb

In [10]:
from azureml.core import Workspace, Environment 
from azureml.core.webservice import AciWebservice
from azureml.core.model import Model, InferenceConfig
from azureml.core.conda_dependencies import CondaDependencies
import sklearn

In [2]:
ws = Workspace.from_config()

In [3]:
import joblib
from sklearn.datasets import load_diabetes
from sklearn.linear_model import Ridge

X, y = load_diabetes(return_X_y=True)
model = Ridge().fit(X, y)

joblib.dump(value=model, filename="sklearn_regression_model.pkl")

['sklearn_regression_model.pkl']

In [4]:
import numpy as np 
from azureml.core import Dataset 

np.savetxt("features.csv", X, delimiter=",")
np.savetxt("labels.csv", y, delimiter=",")

datastore = ws.get_default_datastore()
datastore.upload_files(
    files=["./features.csv", "./labels.csv"], 
    target_path="sklearn-regression/", 
    overwrite=True, 
    show_progress=True
)

"datastore.upload_files" is deprecated after version 1.0.69. Please use "FileDatasetFactory.upload_directory" instead. See Dataset API change notice at https://aka.ms/dataset-deprecation.


Uploading an estimated of 2 files
Uploading ./labels.csv
Uploaded ./labels.csv, 1 files out of an estimated total of 2
Uploading ./features.csv
Uploaded ./features.csv, 2 files out of an estimated total of 2
Uploaded 2 files


$AZUREML_DATAREFERENCE_7f59f12827dd4b249376ed14346f8356

In [5]:
input_dataset = Dataset.Tabular.from_delimited_files(path=(datastore, "sklearn-regression/features.csv"))
output_dataset = Dataset.Tabular.from_delimited_files(path=(datastore, "sklearn-regression/labels.csv"))

In [38]:
# create custom environment
environment = Environment("sklearn-env")
environment.python.conda_dependencies = CondaDependencies.create(
    conda_packages=["pip"],
    pip_packages=[
        "azureml-defaults", 
        "inference-schema[numpy-support]", 
        "joblib",
        "numpy",
        "scikit-learn"
    ],
)

In [ ]:
with open('score.py') as f:
    print(f.read())

In [44]:
# register the model 
model = Model.register(
    workspace=ws,
    model_name="sklearn-regression-model",
    model_path="./sklearn_regression_model.pkl",
    model_framework=Model.Framework.SCIKITLEARN,
    model_framework_version=sklearn.__version__,
    sample_input_dataset=input_dataset,
    sample_output_dataset=output_dataset,
    description="Ridge regression model to predict diabetes progression",
    tags={"area": "diabetes", "type": "regression"}
)

print("Name", model.name)
print("Version", model.version)

Registering model sklearn-regression-model


Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'


Name sklearn-regression-model
Version 5


In [ ]:
service_name = "aci-deploy"

inference_config = InferenceConfig(entry_script="score.py", environment=environment)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

In [ ]:
# test model locally
# package = Model.package(ws, [model], inference_config)
# package.wait_for_creation(show_output=True)

In [45]:
service_name = "aci-deploy"

inference_config = InferenceConfig(entry_script="score.py", environment=environment)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(
    workspace=ws,
    name=service_name,
    models=[model],
    inference_config=inference_config,
    deployment_config=aci_config,
    overwrite=True
)
service.wait_for_deployment(show_output=True)

c:\Users\leopu\OneDrive\Programming\Python\azure\sdk-v1\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  if sys.path[0] == "":


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-04-09 14:36:46+02:00 Creating Container Registry if not exists.
2023-04-09 14:36:46+02:00 Registering the environment.
2023-04-09 14:36:47+02:00 Use the existing image.
2023-04-09 14:36:47+02:00 Generating deployment configuration.
2023-04-09 14:36:48+02:00 Submitting deployment to compute.
2023-04-09 14:36:51+02:00 Checking the status of deployment aci-deploy..
2023-04-09 14:38:22+02:00 Checking the status of inference endpoint aci-deploy.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [46]:
print(service.get_logs())

/bin/bash: /azureml-envs/azureml_6f71df123ef209ef043f25f4eab57899/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_6f71df123ef209ef043f25f4eab57899/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_6f71df123ef209ef043f25f4eab57899/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_6f71df123ef209ef043f25f4eab57899/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2023-04-09T12:38:08,621416500+00:00 - rsyslog/run 
2023-04-09T12:38:08,622721400+00:00 - iot-server/run 
2023-04-09T12:38:08,630982500+00:00 - gunicorn/run 
2023-04-09T12:38:08,632712500+00:00 | gunicorn/run | 
bash: /azureml-envs/azureml_6f71df123ef209ef043f25f4eab57899/lib/libtinfo.so.6: no version information available (required by bash)
2023-04-09T12:38:08,646146900+00:00 | gunicorn/run | ####################################

In [33]:
# use this to delete the service
# service.delete()

In [47]:
# test the deployed model
import json

input_payload = json.dumps({
    "data": X[0:2].tolist(),
    #"method": "predict"
})

output = service.run(input_payload)
print(output)

[182.67357342863968, 90.99902728640282]
